# 🔍 Markov Model + Diagnostic DEL Tăng Liên Tục

Notebook hoàn chỉnh bao gồm:
1. Load data
2. Build transition matrices  
3. Calibration (K values)
4. Forecast
5. **DIAGNOSTIC** - Tại sao DEL tăng sau MOB 24?
6. Giải pháp

---

## 1️⃣ SETUP & IMPORT

In [ ]:
# Setup
import sys
from pathlib import Path
project_root = Path(".").resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

from src.config import CFG, BUCKETS_CANON, BUCKETS_30P, BUCKETS_90P
from src.config import parse_date_column, create_segment_columns, SEGMENT_COLS
from src.config import K_POST_MATURE  # K value cho MOB > TARGET_MOB
from src.data_loader import load_data
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.lifecycle import get_actual_all_vintages_amount
from src.rollrate.calibration_kmob import (
    fit_k_raw, smooth_k, fit_alpha,
    forecast_all_vintages_partial_step,
)

print("✅ Import thành công")

## 2️⃣ LOAD DATA

In [ ]:
# ========== CẤU HÌNH ==========
DATA_PATH = 'C:/Users/User/Projection_PB/Projection_pb/ETB_Parquet_YYYYMM'
MAX_MOB = 36  # Forecast đến MOB 36
# ==============================

df_raw = load_data(DATA_PATH)
df_raw['DISBURSAL_DATE'] = parse_date_column(df_raw['DISBURSAL_DATE'])
df_raw = create_segment_columns(df_raw)

print(f"📊 Data: {len(df_raw):,} rows | {df_raw[CFG['loan']].nunique():,} loans")
print(f"   Products: {df_raw['PRODUCT_TYPE'].unique().tolist()}")

## 3️⃣ BUILD TRANSITION MATRICES

In [ ]:
print("🔨 Building transition matrices...")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)
print(f"✅ {len(matrices_by_mob)} products | {sum(len(m) for m in matrices_by_mob.values())} matrices")

## 4️⃣ CALIBRATION

In [ ]:
print("🔨 Calibrating k and alpha...")

# Actual results
actual_results = get_actual_all_vintages_amount(df_raw)

# DISB_TOTAL map
loan_disb = df_raw.groupby(["PRODUCT_TYPE", "RISK_SCORE", CFG["orig_date"], CFG["loan"]])[CFG["disb"]].first()
disb_total_by_vintage = loan_disb.groupby(level=[0, 1, 2]).sum().to_dict()

# Fit k_raw
k_raw_by_mob, weight_by_mob, _ = fit_k_raw(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=BUCKETS_CANON,
    s30_states=BUCKETS_30P,
    include_co=True,
    denom_mode="disb",
    disb_total_by_vintage=disb_total_by_vintage,
    weight_mode="equal",
    method="wls_reg",
    lambda_k=1e-4,
    k_prior=0.0,
    min_obs=5,
    fallback_k=1.0,
    fallback_weight=0.0,
    return_detail=True,
)

print(f"   K values: {len(k_raw_by_mob)} MOBs")

# Smooth k
mob_min = min(k_raw_by_mob.keys()) if k_raw_by_mob else 0
mob_max = max(k_raw_by_mob.keys()) if k_raw_by_mob else 0
k_smooth_by_mob, _, _ = smooth_k(k_raw_by_mob, weight_by_mob, mob_min, mob_max)

# Fit alpha
alpha, k_final_by_mob, _ = fit_alpha(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=BUCKETS_CANON,
    s30_states=BUCKETS_30P,
    k_smooth_by_mob=k_smooth_by_mob,
    mob_target=min(MAX_MOB, mob_max) if mob_max else MAX_MOB,
    include_co=True,
)

print(f"   Alpha: {alpha:.4f}")
print(f"   K_final: {len(k_final_by_mob)} MOBs")

# ============================
# APPLY K_POST_MATURE (nếu được cấu hình)
# ============================
# K_POST_MATURE: Giá trị K cố định cho MOB > TARGET_MOB (24)
# Mục đích: Giảm slope của DEL curve sau khi mature

TARGET_MOB = 24  # MOB mà portfolio được coi là mature

if K_POST_MATURE is not None:
    print(f"\n🔧 Applying K_POST_MATURE = {K_POST_MATURE} for MOB >= {TARGET_MOB}")
    
    # Lưu K trước khi thay đổi để so sánh
    k_before = {mob: k_final_by_mob.get(mob, 1.0) for mob in range(TARGET_MOB, MAX_MOB + 1)}
    
    # Apply K_POST_MATURE cho MOB >= TARGET_MOB
    for mob in range(TARGET_MOB, MAX_MOB + 1):
        k_final_by_mob[mob] = K_POST_MATURE
    
    # In so sánh
    print(f"   K values comparison (MOB {TARGET_MOB} → {MAX_MOB}):")
    print(f"   {'MOB':<6} {'Before':<10} {'After':<10}")
    print(f"   {'-'*26}")
    for mob in range(TARGET_MOB, min(TARGET_MOB + 5, MAX_MOB + 1)):
        k_old = k_before.get(mob, 1.0)
        k_new = k_final_by_mob.get(mob, 1.0)
        marker = '← TARGET_MOB' if mob == TARGET_MOB else ''
        print(f"   {mob:<6} {k_old:<10.4f} {k_new:<10.4f} {marker}")
    if MAX_MOB > TARGET_MOB + 5:
        print(f"   ...")
        print(f"   {MAX_MOB:<6} {k_before.get(MAX_MOB, 1.0):<10.4f} {k_final_by_mob.get(MAX_MOB, 1.0):<10.4f}")
    
    print(f"\n   ✅ K_POST_MATURE applied: MOB {TARGET_MOB} → {MAX_MOB} = {K_POST_MATURE}")
else:
    print(f"\n   ℹ️  K_POST_MATURE = None, using calibrated K values")

## 5️⃣ FORECAST

In [ ]:
# Forecast với k_final
forecast_results = forecast_all_vintages_partial_step(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=MAX_MOB,
    k_by_mob=k_final_by_mob,
    states=BUCKETS_CANON,
)

print(f"✅ Forecast: {len(forecast_results)} cohorts")

---

# 🔍 DIAGNOSTIC: TẠI SAO DEL TĂNG SAU MOB 24?

---

## 6️⃣ DIAGNOSTIC 1: Kiểm Tra K Values

In [ ]:
print("="*80)
print("1️⃣ KIỂM TRA K VALUES")
print("="*80)

print("\n   MOB  |  K value  |  Status")
print("   -----|-----------|----------")

k_issues = []
for mob in range(20, 37):
    k = k_final_by_mob.get(mob, 1.0)
    
    if k > 0.9:
        status = "❌ Rất cao"
        if mob >= 25:
            k_issues.append(f"MOB {mob}: k={k:.3f}")
    elif k > 0.7:
        status = "⚠️ Cao"
    elif k > 0.5:
        status = "✅ Trung bình"
    else:
        status = "✅ Thấp"
    
    print(f"   {mob:4d} | {k:9.3f} | {status}")

print("\n" + "-"*80)

if k_issues:
    print(f"\n❌ PHÁT HIỆN VẤN ĐỀ: {len(k_issues)} MOBs có K quá cao")
    for issue in k_issues:
        print(f"   - {issue}")
    print(f"\n💡 Giải thích:")
    print(f"   - K cao → Model tin Markov quá nhiều")
    print(f"   - Markov gây movement → DEL tăng")
    print(f"   - Cần giảm K xuống ~0.3 cho MOB 25+")
else:
    print(f"\n✅ K values hợp lý (không quá cao)")

## 7️⃣ DIAGNOSTIC 2: Kiểm Tra Fallback Usage

In [ ]:
print("\n" + "="*80)
print("2️⃣ KIỂM TRA % COHORTS DÙNG FALLBACK Ở MOB 24")
print("="*80)

total_cohorts = 0
fallback_cohorts = 0
fallback_details = []
fallback_pct = 0.0

for prod_str in matrices_by_mob.keys():
    if 24 in matrices_by_mob[prod_str]:
        for score_str in matrices_by_mob[prod_str][24].keys():
            total_cohorts += 1
            is_fallback = matrices_by_mob[prod_str][24][score_str].get("is_fallback", False)
            if is_fallback:
                fallback_cohorts += 1
                reason = matrices_by_mob[prod_str][24][score_str].get("reason", "unknown")
                fallback_details.append((prod_str, score_str, reason))

print(f"\n   Tổng cohorts: {total_cohorts}")
print(f"   Cohorts dùng fallback ở MOB 24: {fallback_cohorts}")

if total_cohorts > 0:
    fallback_pct = fallback_cohorts / total_cohorts * 100
    print(f"   Tỷ lệ: {fallback_pct:.1f}%")
    
    print("\n" + "-"*80)
    
    if fallback_pct > 30:
        print(f"\n❌ PHÁT HIỆN VẤN ĐỀ: Quá nhiều cohorts dùng fallback!")
        print(f"\n💡 Giải thích:")
        print(f"   - Các cohorts này dùng parent fallback (có movement cao)")
        print(f"   - Parent fallback tổng hợp MOB 1-24 (MOB sớm có rates cao)")
        print(f"   - Gây DEL tăng ở MOB 25+")
        
        print(f"\n   Chi tiết cohorts dùng fallback (top 10):")
        for prod, score, reason in fallback_details[:10]:
            print(f"      - {prod}/{score}: {reason}")
        if len(fallback_details) > 10:
            print(f"      ... và {len(fallback_details)-10} cohorts khác")
    elif fallback_pct > 10:
        print(f"\n⚠️ Có một số cohorts dùng fallback ({fallback_pct:.1f}%)")
    else:
        print(f"\n✅ Ít cohorts dùng fallback ({fallback_pct:.1f}%)")
else:
    print("\n⚠️ Không có cohorts nào ở MOB 24")

## 8️⃣ DIAGNOSTIC 3: So Sánh P_24 vs Parent Fallback

In [ ]:
print("\n" + "="*80)
print("3️⃣ SO SÁNH P_24 vs PARENT FALLBACK")
print("="*80)

# Tìm 1 cohort để test (không dùng fallback)
test_prod = None
test_score = None

for prod_str in matrices_by_mob.keys():
    if 24 in matrices_by_mob[prod_str]:
        for score_str in matrices_by_mob[prod_str][24].keys():
            if not matrices_by_mob[prod_str][24][score_str].get("is_fallback", False):
                test_prod = prod_str
                test_score = score_str
                break
    if test_prod:
        break

if test_prod and test_score:
    try:
        P_24 = matrices_by_mob[test_prod][24][test_score]["P"]
        key_parent = (test_prod, test_score)
        
        if key_parent in parent_fallback:
            P_parent = parent_fallback[key_parent]
            
            print(f"\n   Test cohort: {test_prod}/{test_score}")
            
            # So sánh DPD0 → DEL30+
            if "DPD0" in P_24.index and "DPD0" in P_parent.index:
                del30_states = ["DPD30+", "DPD60+", "DPD90+", "DPD120+", "DPD180+", "WRITEOFF"]
                
                p24_to_del30 = sum(P_24.loc["DPD0", s] for s in del30_states if s in P_24.columns)
                parent_to_del30 = sum(P_parent.loc["DPD0", s] for s in del30_states if s in P_parent.columns)
                
                print(f"\n   DPD0 → DEL30+ comparison:")
                print(f"   P_24:    {p24_to_del30:.4f} ({p24_to_del30*100:.2f}%)")
                print(f"   Parent:  {parent_to_del30:.4f} ({parent_to_del30*100:.2f}%)")
                print(f"   Diff:    {parent_to_del30 - p24_to_del30:+.4f} ({(parent_to_del30 - p24_to_del30)*100:+.2f}%)")
                
                print("\n" + "-"*80)
                
                if parent_to_del30 > p24_to_del30 * 1.5:
                    print(f"\n✅ XÁC NHẬN: Parent fallback có movement cao hơn P_24 nhiều")
                    print(f"   - P_24 ổn định hơn (portfolio đã mature ở MOB 24)")
                    print(f"   - Parent fallback có movement cao (tổng hợp MOB sớm)")
                elif parent_to_del30 > p24_to_del30:
                    print(f"\n✅ Parent fallback hơi cao hơn P_24")
                else:
                    print(f"\n⚠️ P_24 cao hơn hoặc bằng parent fallback")
            else:
                print("\n⚠️ Không tìm thấy DPD0 trong matrices")
        else:
            print(f"\n⚠️ Không tìm thấy parent fallback cho {test_prod}/{test_score}")
    except Exception as e:
        print(f"\n⚠️ Lỗi khi so sánh: {e}")
else:
    print("\n⚠️ Không tìm thấy cohort nào để test")

## 9️⃣ DIAGNOSTIC 4: Phân Tích Cohorts

In [ ]:
print("\n" + "="*80)
print("4️⃣ PHÂN TÍCH TỪNG COHORT")
print("="*80)

cohort_count = 0
increasing_cohorts = []
flat_cohorts = []

# Test 10 cohorts đầu
try:
    for cohort_key in list(forecast_results.keys())[:10]:
        forecast = forecast_results[cohort_key]
        disb_total = disb_total_by_vintage.get(cohort_key, 1.0)
        
        # Tính DEL30+ at MOB 24 and 30
        if 24 in forecast and 30 in forecast:
            try:
                del30_24 = forecast[24][BUCKETS_30P].sum() / disb_total
                del30_30 = forecast[30][BUCKETS_30P].sum() / disb_total
                slope = (del30_30 - del30_24) / 6
                
                cohort_count += 1
                
                if abs(slope) > 0.001:
                    increasing_cohorts.append((cohort_key, slope))
                else:
                    flat_cohorts.append((cohort_key, slope))
            except Exception as e:
                print(f"   ⚠️ Lỗi khi phân tích cohort {cohort_key}: {e}")
                continue
    
    print(f"\n   Đã phân tích {cohort_count} cohorts:")
    print(f"   - Cohorts tăng (slope > 0.001): {len(increasing_cohorts)}")
    print(f"   - Cohorts flat (slope ≤ 0.001): {len(flat_cohorts)}")
    
    if len(increasing_cohorts) > 0:
        print(f"\n   Top cohorts tăng mạnh:")
        for cohort_key, slope in sorted(increasing_cohorts, key=lambda x: abs(x[1]), reverse=True)[:5]:
            prod, score, vintage = cohort_key
            print(f"      - {prod}/{score}/{vintage}: slope = {slope:.6f} ({slope*100:.4f}%/month)")
            
            # Kiểm tra cohort này dùng fallback không
            prod_str = str(prod)
            score_str = str(score)
            if prod_str in matrices_by_mob and 24 in matrices_by_mob[prod_str]:
                if score_str in matrices_by_mob[prod_str][24]:
                    is_fallback = matrices_by_mob[prod_str][24][score_str].get("is_fallback", False)
                    if is_fallback:
                        print(f"        → ❌ Cohort này dùng FALLBACK ở MOB 24!")
                    else:
                        print(f"        → ✅ Cohort này dùng P_24 thật")
    
    print("\n" + "-"*80)
    
    if len(increasing_cohorts) > len(flat_cohorts):
        print(f"\n❌ PHÁT HIỆN VẤN ĐỀ: Nhiều cohorts vẫn tăng sau MOB 24")
    else:
        print(f"\n✅ Đa số cohorts đã flatten sau MOB 24")
        
except Exception as e:
    print(f"\n⚠️ Lỗi khi phân tích cohorts: {e}")
    increasing_cohorts = []
    flat_cohorts = []

## 🔟 KẾT LUẬN VÀ KHUYẾN NGHỊ

In [ ]:
print("\n" + "="*80)
print("KẾT LUẬN")
print("="*80)

conclusions = []

if k_issues:
    conclusions.append("❌ K values quá cao ở MOB 25+ → Tin Markov quá nhiều")

if total_cohorts > 0 and fallback_pct > 30:
    conclusions.append("❌ Nhiều cohorts dùng parent fallback ở MOB 24 → Movement cao")

if len(increasing_cohorts) > len(flat_cohorts):
    conclusions.append("❌ Nhiều cohorts vẫn tăng sau MOB 24")

if not conclusions:
    conclusions.append("✅ Không phát hiện vấn đề rõ ràng")
    conclusions.append("   → Có thể là aggregation effect hoặc weighting")

for conclusion in conclusions:
    print(f"\n{conclusion}")

print("\n" + "="*80)
print("KHUYẾN NGHỊ")
print("="*80)

if k_issues:
    print("\n1️⃣ GIẢM K Ở MOB 25+")
    print("   → Chạy cell 'Giải pháp 1' bên dưới")

if total_cohorts > 0 and fallback_pct > 30:
    print("\n2️⃣ TĂNG MIN_OBS/MIN_EAD")
    print("   → Xem hướng dẫn trong cell 'Giải pháp 2'")

if len(increasing_cohorts) > 0:
    print("\n3️⃣ KIỂM TRA CHI TIẾT COHORTS TĂNG MẠNH")
    print("   → Xem có pattern chung không (product, score, vintage)")

print("\n" + "="*80)

---

# GIẢI PHÁP

---

## Giải Pháp 1: Cap K ở MOB 25+

**Chỉ chạy cell này nếu diagnostic cho thấy: ❌ K values quá cao**

In [ ]:
print("🔧 ÁP DỤNG GIẢI PHÁP 1: Cap K ở MOB 25+")
print("="*80)

print("\nK values TRƯỚC KHI CAP:")
for mob in range(24, 37):
    k_val = k_final_by_mob.get(mob, 1.0)
    status = "❌ Cao" if k_val > 0.9 else "✅ OK"
    print(f"  MOB {mob}: {k_val:.3f} {status}")

# Cap K
print("\n🔧 Đang cap K...")
for mob in range(25, 37):
    if mob in k_final_by_mob:
        k_final_by_mob[mob] = min(k_final_by_mob[mob], 0.3)
    else:
        k_final_by_mob[mob] = 0.3

print("\nK values SAU KHI CAP:")
for mob in range(24, 37):
    k_val = k_final_by_mob.get(mob, 1.0)
    print(f"  MOB {mob}: {k_val:.3f} ✅")

print("\n" + "="*80)
print("✅ ĐÃ CAP K!")
print("="*80)
print("\n💡 Bước tiếp theo:")
print("   1. Re-run cell 5 (Forecast) với k_final_by_mob đã được cap")
print("   2. Re-run các cells diagnostic (6-10) để verify")
print("   3. Kiểm tra lại kết quả")

## Giải Pháp 2: Tăng MIN_OBS/MIN_EAD

**Chỉ áp dụng nếu diagnostic cho thấy: ❌ Nhiều cohorts dùng fallback**

In [ ]:
print("📝 GIẢI PHÁP 2: Tăng MIN_OBS/MIN_EAD")
print("="*80)
print("\n⚠️  Giải pháp này yêu cầu sửa file src/config.py")
print("\n📝 Các bước:")
print("   1. Mở file: src/config.py")
print("   2. Tìm dòng: MIN_OBS = 100")
print("   3. Sửa thành: MIN_OBS = 200")
print("   4. Tìm dòng: MIN_EAD = 1e2")
print("   5. Sửa thành: MIN_EAD = 5e2")
print("   6. Save file")
print("   7. Restart kernel và chạy lại từ đầu")
print("\n💡 Hoặc xem chi tiết trong: HUONG_DAN_CHAY_DIAGNOSTIC.md")
print("="*80)

## 1️⃣1️⃣ SO SÁNH P_23 MOVEMENT vs FORECAST SLOPE

**Mục đích**: So sánh P_23 movement (từ transition matrix) với forecast slope (MOB 23 → 29) để hiểu:
1. P_23 có movement bao nhiêu?
2. Forecast slope có match với P_23 không?
3. Nếu match → K=1.0 đang work đúng, vấn đề là P_23 có movement
4. Nếu không match → Có bug trong forecast logic

In [ ]:
from compare_p24_vs_forecast import compare_p24_vs_forecast

print("🔬 SO SÁNH P_23 MOVEMENT vs FORECAST SLOPE")
print("="*100)

df_comparison = compare_p24_vs_forecast(
    matrices_by_mob=matrices_by_mob,
    forecast_results=forecast_results,
    actual_results=actual_results,
    disb_total_by_vintage=disb_total_by_vintage,
    buckets_30p=BUCKETS_30P,
    target_mob=23,  # Dùng MOB 23 thay vì 24
    forecast_mob_end=29  # Forecast đến MOB 29
)

if df_comparison is not None:
    print("\n✅ Đã tạo df_comparison")
    print("\n💡 Bạn có thể:")
    print("   - Xem top cohorts: df_comparison.head(20)")
    print("   - Export: df_comparison.to_excel('comparison_p23_vs_forecast.xlsx', index=False)")
    print("   - Lọc cohorts có diff lớn: df_comparison[df_comparison['diff_pct'] > 0.5]")
else:
    print("\n❌ Không tìm thấy data")

## 1️⃣2️⃣ PHÂN TÍCH P_23 vs PARENT FALLBACK

**Mục đích**: Hiểu tại sao forecast cao hơn P_23 tới 1400 lần:
1. P_23 có movement bao nhiêu?
2. Parent fallback có movement bao nhiêu?
3. Cohorts nào đang dùng parent fallback?
4. Parent fallback có cao hơn P_23 không?

In [ ]:
from analyze_p23_vs_parent import analyze_p23_vs_parent

print("🔬 PHÂN TÍCH P_23 vs PARENT FALLBACK")
print("="*100)

df_p23_parent = analyze_p23_vs_parent(
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    buckets_30p=BUCKETS_30P
)

if df_p23_parent is not None:
    print("\n✅ Đã tạo df_p23_parent")
    print("\n💡 Bạn có thể:")
    print("   - Xem top cohorts: df_p23_parent.head(20)")
    print("   - Export: df_p23_parent.to_excel('p23_vs_parent.xlsx', index=False)")
    print("   - Lọc cohorts dùng fallback: df_p23_parent[df_p23_parent['is_fallback']]")
else:
    print("\n❌ Không tìm thấy data")

## 1️⃣3️⃣ PHÂN TÍCH K VALUES CHI TIẾT

**Mục đích**: Trả lời câu hỏi:
- K trước MOB 24 là bao nhiêu?
- K sau MOB 24 là bao nhiêu?
- Có K jumps lớn không?
- Tại sao K là vấn đề nếu transitions đã ổn định?

In [ ]:
print("=" * 100)
print("🔍 PHÂN TÍCH K VALUES CHI TIẾT")
print("=" * 100)

# 1. Show K values table
print("\n📊 K VALUES THEO MOB:")
print("\n   MOB  |  K_raw  |  K_smooth  |  K_final  |  Change  |  Status")
print("   -----|---------|------------|-----------|----------|----------")

prev_k = None
k_jumps = []

for mob in range(1, 37):
    k_raw = k_raw_by_mob.get(mob, np.nan)
    k_smooth = k_smooth_by_mob.get(mob, np.nan)
    k_final = k_final_by_mob.get(mob, 1.0)
    
    if prev_k is not None and not np.isnan(k_final):
        change = k_final - prev_k
        change_str = f"{change:+.3f}"
        
        # Detect jumps
        if abs(change) > 0.2:
            k_jumps.append((mob, prev_k, k_final, change))
            status = "⚠️ JUMP!"
        elif k_final > 0.9:
            status = "❌ Rất cao"
        elif k_final > 0.7:
            status = "⚠️ Cao"
        else:
            status = "✅ OK"
    else:
        change_str = "N/A"
        status = "✅ Start"
    
    k_raw_str = f"{k_raw:.3f}" if not np.isnan(k_raw) else "N/A"
    k_smooth_str = f"{k_smooth:.3f}" if not np.isnan(k_smooth) else "N/A"
    
    print(f"   {mob:4d} | {k_raw_str:7s} | {k_smooth_str:10s} | {k_final:9.3f} | {change_str:8s} | {status}")
    
    if not np.isnan(k_final):
        prev_k = k_final

# 2. Statistics
print("\n" + "=" * 100)
print("📊 THỐNG KÊ K VALUES")
print("=" * 100)

# K values before and after MOB 24
k_before_24 = [k_final_by_mob.get(m, np.nan) for m in range(12, 24)]
k_after_24 = [k_final_by_mob.get(m, np.nan) for m in range(24, 30)]

k_before_24 = [k for k in k_before_24 if not np.isnan(k)]
k_after_24 = [k for k in k_after_24 if not np.isnan(k)]

if k_before_24 and k_after_24:
    avg_before = np.mean(k_before_24)
    avg_after = np.mean(k_after_24)
    
    print(f"\n   K trung bình TRƯỚC MOB 24 (MOB 12-23): {avg_before:.3f}")
    print(f"   K trung bình SAU MOB 24 (MOB 24-29):   {avg_after:.3f}")
    print(f"   Chênh lệch:                             {avg_after - avg_before:+.3f} ({(avg_after/avg_before - 1)*100:+.1f}%)")
    
    if avg_after > avg_before * 1.2:
        print(f"\n   ❌ K SAU MOB 24 CAO HƠN TRƯỚC MOB 24 NHIỀU!")
        print(f"   → Đây là lý do slope SAU mature cao hơn slope TRƯỚC mature")
        print(f"   → Ngay cả khi P_m ổn định, K tăng cũng làm slope tăng")
    else:
        print(f"\n   ✅ K không thay đổi nhiều")
else:
    print("\n   ⚠️ Không đủ data để so sánh")

# 3. K jumps
if k_jumps:
    print(f"\n   ❌ PHÁT HIỆN {len(k_jumps)} K JUMPS (>0.2):")
    for mob, k_before, k_after, change in k_jumps:
        print(f"      - MOB {mob}: {k_before:.3f} → {k_after:.3f} (change: {change:+.3f})")
else:
    print(f"\n   ✅ Không có K jumps lớn (>0.2)")

# 4. Explanation
print("\n" + "=" * 100)
print("💡 GIẢI THÍCH: TẠI SAO K LÀ VẤN ĐỀ?")
print("=" * 100)

print("\n   Công thức forecast:")
print("   v_{m+1} = v_m + k_m * (v_hat - v_m)")
print("   where v_hat = v_m @ P_m")

print("\n   Nếu P_m có movement (ví dụ: DPD0 → DEL30+ = 0.0004%):")

if k_before_24 and k_after_24:
    avg_before = np.mean(k_before_24)
    avg_after = np.mean(k_after_24)
    
    movement = 0.0004  # P_23 movement từ kết quả trước
    
    forecast_before = avg_before * movement
    forecast_after = avg_after * movement
    
    print(f"\n   TRƯỚC MOB 24 (K = {avg_before:.3f}):")
    print(f"      Forecast movement = {avg_before:.3f} * {movement:.4f}% = {forecast_before:.6f}%")
    
    print(f"\n   SAU MOB 24 (K = {avg_after:.3f}):")
    print(f"      Forecast movement = {avg_after:.3f} * {movement:.4f}% = {forecast_after:.6f}%")
    
    print(f"\n   Chênh lệch: {forecast_after - forecast_before:.6f}% ({(forecast_after/forecast_before - 1)*100:+.1f}%)")
    
    if forecast_after > forecast_before * 1.2:
        print(f"\n   ❌ FORECAST MOVEMENT SAU MOB 24 CAO HƠN TRƯỚC MOB 24!")
        print(f"   → Đây là lý do slope tăng")
        print(f"   → Ngay cả khi P_m ổn định, K tăng cũng làm slope tăng")

# 5. Conclusion
print("\n" + "=" * 100)
print("KẾT LUẬN")
print("=" * 100)

print("\n   'Ổn định' có 2 nghĩa:")
print("   1. P_m không thay đổi theo MOB (P_23 ≈ P_24 ≈ P_25)")
print("   2. P_m không gây movement (v_hat ≈ v_m)")

print("\n   P_m có thể 'ổn định' theo nghĩa 1 nhưng vẫn có movement!")
print("   → K quyết định bao nhiêu % movement được áp dụng")
print("   → K tăng → Forecast movement tăng → Slope tăng")

if k_before_24 and k_after_24:
    avg_before = np.mean(k_before_24)
    avg_after = np.mean(k_after_24)
    
    if avg_after > avg_before * 1.2:
        print(f"\n   ❌ K SAU MOB 24 ({avg_after:.3f}) cao hơn TRƯỚC MOB 24 ({avg_before:.3f})")
        print(f"   → Đây là lý do slope SAU mature cao hơn slope TRƯỚC mature")
        print(f"   → Giải pháp: Giảm K sau MOB 24 xuống {avg_before:.3f}")

print("\n" + "=" * 100)

## 1️⃣4️⃣ GIẢI THUYẾT 2: KIỂM TRA TRANSITION STABILITY

**Mục đích**: Kiểm tra xem transitions có thực sự ổn định không?
- P_23, P_24, P_25 có movement bao nhiêu?
- Movement có tăng sau MOB 24 không?
- Có cohorts nào có movement cao bất thường không?

In [ ]:
print("=" * 100)
print("🔍 GIẢI THUYẾT 2: KIỂM TRA TRANSITION STABILITY")
print("=" * 100)

# Find a test cohort (not using fallback)
test_prod = None
test_score = None

for prod_str in matrices_by_mob.keys():
    if 23 in matrices_by_mob[prod_str]:
        for score_str in matrices_by_mob[prod_str][23].keys():
            if not matrices_by_mob[prod_str][23][score_str].get("is_fallback", False):
                test_prod = prod_str
                test_score = score_str
                break
    if test_prod:
        break

if test_prod and test_score:
    print(f"\n📊 Test cohort: {test_prod}/{test_score}")
    print("\n   MOB  |  DPD0→DEL30+  |  Change  |  Status  |  Fallback")
    print("   -----|---------------|----------|----------|----------")
    
    prev_rate = None
    movements = []
    
    for mob in range(20, 31):
        if mob not in matrices_by_mob[test_prod]:
            continue
        
        if test_score not in matrices_by_mob[test_prod][mob]:
            continue
        
        P = matrices_by_mob[test_prod][mob][test_score]["P"]
        is_fallback = matrices_by_mob[test_prod][mob][test_score].get("is_fallback", False)
        
        if "DPD0" not in P.index:
            continue
        
        # Calculate DPD0 → DEL30+ rate
        del30_states = [s for s in BUCKETS_30P if s in P.columns]
        rate = sum(P.loc["DPD0", s] for s in del30_states)
        
        if prev_rate is not None:
            change = rate - prev_rate
            change_str = f"{change:+.6f}"
            movements.append((mob, change))
            
            if abs(change) > 0.001:
                status = "⚠️ Movement"
            else:
                status = "✅ Stable"
        else:
            change_str = "N/A"
            status = "✅ Start"
        
        fallback_str = "❌ Yes" if is_fallback else "✅ No"
        print(f"   {mob:4d} | {rate:13.6f} | {change_str:8s} | {status:8s} | {fallback_str}")
        prev_rate = rate
    
    # Statistics
    if movements:
        print("\n" + "-" * 100)
        
        movements_before_24 = [abs(m[1]) for m in movements if m[0] < 24]
        movements_after_24 = [abs(m[1]) for m in movements if m[0] >= 24]
        
        avg_movement = np.mean([abs(m[1]) for m in movements])
        max_movement = max([abs(m[1]) for m in movements])
        
        print(f"\n📊 THỐNG KÊ MOVEMENT:")
        print(f"   - Average movement (all):     {avg_movement:.6f} ({avg_movement*100:.4f}%)")
        print(f"   - Max movement:               {max_movement:.6f} ({max_movement*100:.4f}%)")
        
        if movements_before_24 and movements_after_24:
            avg_before = np.mean(movements_before_24)
            avg_after = np.mean(movements_after_24)
            print(f"   - Average movement BEFORE 24: {avg_before:.6f} ({avg_before*100:.4f}%)")
            print(f"   - Average movement AFTER 24:  {avg_after:.6f} ({avg_after*100:.4f}%)")
            print(f"   - Chênh lệch:                 {avg_after - avg_before:+.6f} ({(avg_after/avg_before - 1)*100:+.1f}%)")
        
        print("\n" + "-" * 100)
        
        if avg_movement > 0.001:
            print(f"\n❌ TRANSITIONS KHÔNG ỔN ĐỊNH!")
            print(f"   - Average movement {avg_movement*100:.4f}% > 0.1%")
            print(f"   - Đây có thể là lý do DEL tăng!")
            print(f"\n💡 Giải thích:")
            print(f"   - P_m có movement cao → Forecast sẽ tăng")
            print(f"   - Ngay cả khi K ổn định, P_m movement cao cũng gây DEL tăng")
        else:
            print(f"\n✅ Transitions ổn định (movement < 0.1%)")
            print(f"   - P_m movement thấp → Không phải nguyên nhân chính")
else:
    print("\n⚠️ Không tìm thấy cohort để test")

print("\n" + "=" * 100)

## 1️⃣5️⃣ GIẢI THUYẾT 3: KIỂM TRA FALLBACK USAGE

**Mục đích**: Kiểm tra xem % cohorts dùng fallback có tăng sau MOB 24 không?
- % fallback ở MOB 20-30
- Có tăng đột ngột sau MOB 24 không?
- Parent fallback có movement cao hơn P_m không?

In [ ]:
print("=" * 100)
print("🔍 GIẢI THUYẾT 3: KIỂM TRA FALLBACK USAGE")
print("=" * 100)

print("\n📊 % COHORTS DÙNG FALLBACK THEO MOB:")
print("\n   MOB  |  Total  |  Fallback  |  %     |  Status")
print("   -----|---------|------------|--------|----------")

fallback_by_mob = {}

for mob in range(20, 31):
    total_cohorts = 0
    fallback_cohorts = 0
    
    for prod_str in matrices_by_mob.keys():
        if mob in matrices_by_mob[prod_str]:
            for score_str in matrices_by_mob[prod_str][mob].keys():
                total_cohorts += 1
                is_fallback = matrices_by_mob[prod_str][mob][score_str].get("is_fallback", False)
                if is_fallback:
                    fallback_cohorts += 1
    
    if total_cohorts > 0:
        fallback_pct = fallback_cohorts / total_cohorts * 100
        fallback_by_mob[mob] = fallback_pct
        
        if fallback_pct > 50:
            status = "❌ Rất cao"
        elif fallback_pct > 30:
            status = "⚠️ Cao"
        else:
            status = "✅ OK"
        
        print(f"   {mob:4d} | {total_cohorts:7d} | {fallback_cohorts:10d} | {fallback_pct:5.1f}% | {status}")

# Statistics
if fallback_by_mob:
    print("\n" + "-" * 100)
    
    fallback_before_24 = [fallback_by_mob[m] for m in fallback_by_mob.keys() if m < 24]
    fallback_after_24 = [fallback_by_mob[m] for m in fallback_by_mob.keys() if m >= 24]
    
    if fallback_before_24 and fallback_after_24:
        avg_before = np.mean(fallback_before_24)
        avg_after = np.mean(fallback_after_24)
        
        print(f"\n📊 THỐNG KÊ:")
        print(f"   - Average % fallback BEFORE 24: {avg_before:.1f}%")
        print(f"   - Average % fallback AFTER 24:  {avg_after:.1f}%")
        print(f"   - Chênh lệch:                   {avg_after - avg_before:+.1f}% ({(avg_after/avg_before - 1)*100:+.1f}%)")
        
        print("\n" + "-" * 100)
        
        if avg_after > avg_before * 1.2:
            print(f"\n❌ % FALLBACK TĂNG ĐỘT NGỘT SAU MOB 24!")
            print(f"   - % fallback tăng {(avg_after/avg_before - 1)*100:.1f}%")
            print(f"   - Parent fallback có movement cao hơn P_m")
            print(f"   - Đây có thể là lý do DEL tăng!")
            print(f"\n💡 Giải pháp:")
            print(f"   - Tăng MIN_OBS để giảm % fallback")
            print(f"   - Hoặc giảm K cho cohorts dùng fallback")
        else:
            print(f"\n✅ % Fallback không tăng nhiều sau MOB 24")
            print(f"   - Không phải nguyên nhân chính")

print("\n" + "=" * 100)

## 1️⃣6️⃣ TÓM TẮT: CẢ 3 GIẢI THUYẾT

**Mục đích**: Tổng hợp kết quả từ cả 3 giải thuyết để xác định nguyên nhân chính.

In [ ]:
print("=" * 100)
print("📊 TÓM TẮT: CẢ 3 GIẢI THUYẾT")
print("=" * 100)

# Hypothesis 1: K values
print("\n1️⃣ GIẢI THUYẾT 1: K VALUES")
print("-" * 100)

k_before_24 = [k_final_by_mob.get(m, np.nan) for m in range(12, 24)]
k_after_24 = [k_final_by_mob.get(m, np.nan) for m in range(24, 30)]
k_before_24 = [k for k in k_before_24 if not np.isnan(k)]
k_after_24 = [k for k in k_after_24 if not np.isnan(k)]

hypothesis_1_result = "⚠️ Không đủ data"
if k_before_24 and k_after_24:
    avg_k_before = np.mean(k_before_24)
    avg_k_after = np.mean(k_after_24)
    k_change_pct = (avg_k_after / avg_k_before - 1) * 100
    
    print(f"   K trung bình TRƯỚC MOB 24: {avg_k_before:.3f}")
    print(f"   K trung bình SAU MOB 24:   {avg_k_after:.3f}")
    print(f"   Chênh lệch:                {avg_k_after - avg_k_before:+.3f} ({k_change_pct:+.1f}%)")
    
    if avg_k_after > avg_k_before * 1.2:
        hypothesis_1_result = "❌ K TĂNG CAO (>20%)"
        print(f"\n   {hypothesis_1_result}")
        print(f"   → Đây có thể là nguyên nhân chính!")
    else:
        hypothesis_1_result = "✅ K không thay đổi nhiều"
        print(f"\n   {hypothesis_1_result}")

# Hypothesis 2: Transition stability
print("\n2️⃣ GIẢI THUYẾT 2: TRANSITION STABILITY")
print("-" * 100)

hypothesis_2_result = "⚠️ Chưa kiểm tra"
if 'movements' in locals() and movements:
    avg_movement = np.mean([abs(m[1]) for m in movements])
    print(f"   Average movement: {avg_movement:.6f} ({avg_movement*100:.4f}%)")
    
    if avg_movement > 0.001:
        hypothesis_2_result = "❌ TRANSITIONS KHÔNG ỔN ĐỊNH (>0.1%)"
        print(f"\n   {hypothesis_2_result}")
        print(f"   → Đây có thể là nguyên nhân chính!")
    else:
        hypothesis_2_result = "✅ Transitions ổn định"
        print(f"\n   {hypothesis_2_result}")
else:
    print(f"   {hypothesis_2_result}")

# Hypothesis 3: Fallback usage
print("\n3️⃣ GIẢI THUYẾT 3: FALLBACK USAGE")
print("-" * 100)

hypothesis_3_result = "⚠️ Không đủ data"
if fallback_by_mob:
    fallback_before_24 = [fallback_by_mob[m] for m in fallback_by_mob.keys() if m < 24]
    fallback_after_24 = [fallback_by_mob[m] for m in fallback_by_mob.keys() if m >= 24]
    
    if fallback_before_24 and fallback_after_24:
        avg_fb_before = np.mean(fallback_before_24)
        avg_fb_after = np.mean(fallback_after_24)
        fb_change_pct = (avg_fb_after / avg_fb_before - 1) * 100
        
        print(f"   % fallback TRƯỚC MOB 24: {avg_fb_before:.1f}%")
        print(f"   % fallback SAU MOB 24:   {avg_fb_after:.1f}%")
        print(f"   Chênh lệch:              {avg_fb_after - avg_fb_before:+.1f}% ({fb_change_pct:+.1f}%)")
        
        if avg_fb_after > avg_fb_before * 1.2:
            hypothesis_3_result = "❌ FALLBACK TĂNG CAO (>20%)"
            print(f"\n   {hypothesis_3_result}")
            print(f"   → Đây có thể là nguyên nhân chính!")
        else:
            hypothesis_3_result = "✅ Fallback không tăng nhiều"
            print(f"\n   {hypothesis_3_result}")
else:
    print(f"   {hypothesis_3_result}")

# Final conclusion
print("\n" + "=" * 100)
print("KẾT LUẬN CUỐI CÙNG")
print("=" * 100)

print(f"\n   1️⃣ K values:            {hypothesis_1_result}")
print(f"   2️⃣ Transition stability: {hypothesis_2_result}")
print(f"   3️⃣ Fallback usage:       {hypothesis_3_result}")

# Determine primary cause
causes = []
if "❌" in hypothesis_1_result:
    causes.append("K values tăng")
if "❌" in hypothesis_2_result:
    causes.append("Transitions không ổn định")
if "❌" in hypothesis_3_result:
    causes.append("Fallback usage tăng")

print("\n" + "-" * 100)

if causes:
    print(f"\n❌ NGUYÊN NHÂN CHÍNH: {', '.join(causes)}")
    print(f"\n💡 GIẢI PHÁP:")
    
    if "K values tăng" in causes:
        print(f"   1. Giảm K sau MOB 24 (xem cell 'Giải pháp 1')")
    
    if "Transitions không ổn định" in causes:
        print(f"   2. Tăng MIN_OBS để lọc cohorts không ổn định")
    
    if "Fallback usage tăng" in causes:
        print(f"   3. Tăng MIN_OBS để giảm % fallback")
else:
    print(f"\n✅ Không phát hiện nguyên nhân rõ ràng")
    print(f"   → Có thể là aggregation effect hoặc weighting")
    print(f"   → Cần kiểm tra thêm chi tiết từng cohort")

print("\n" + "=" * 100)

---

## ✅ HOÀN THÀNH!

### Tóm Tắt:

1. ✅ Load data và build transition matrices
2. ✅ Calibration (K values)
3. ✅ Forecast
4. ✅ Diagnostic - Xác định vấn đề
5. ✅ Áp dụng giải pháp
6. ✅ Re-run và verify

---